In [3]:
import pydicom
import numpy as np
from PIL import Image

In [ ]:
import os
from pathlib import Path

# Get the current file's directory
current_dir = Path().absolute()

# Set working directory to parent directory (one level up)
parent_dir = current_dir.parent
os.chdir(parent_dir)

# Print confirmation
print(f"Working directory set to: {os.getcwd()}")

In [1]:
import json

# Open and read the JSON file
with open('MR_Yeni (1).json', 'r') as file:
    data = json.load(file)

# Print the data
data

[{'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.31.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.32.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.33.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.5.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.6.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.7.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.3.5.dcm',
  'Modality': 'MR',
  'Less

In [11]:
count={}

for item in data:
    if(item["ImageId"].split(".")[1] not in count):
        count[item["ImageId"].split(".")[1]]=1
    count[item["ImageId"].split(".")[1]]+=int(item["ImageId"].split(".")[1])

count

{'0': 1,
 '1': 1419,
 '3': 1486,
 '2': 2333,
 '4': 745,
 '5': 481,
 '6': 439,
 '10': 71,
 '7': 421,
 '13': 157,
 '14': 85,
 '11': 34,
 '8': 177}

In [4]:

# 1. Read the DICOM
ds = pydicom.dcmread("İskemi_veri kümesi/Diğer (Normal ve Kronik İskemi Bulgu Kesitleri)/103618.dcm")
ds2 = pydicom.dcmread("TeknofestDCM_Anon/Diğer(Normal kesit veya kronik iskemik bulgular)/96358.dcm")

In [6]:
def read_dicom(path):

    ds = pydicom.dcmread(path)
    # 2. Convert to pixel array
    arr=ds.pixel_array
    # 3. (Optional) Apply rescale slope/intercept
    slope = getattr(ds, 'RescaleSlope', 1.0)
    intercept = getattr(ds, 'RescaleIntercept', 0.0)
    arr = arr * slope + intercept

    # 4. Windowing (convert to 0–255)
    #    If the DICOM has WindowCenter/Width tags, use them:
    if ('WindowCenter' in ds) and ('WindowWidth' in ds):
        center = ds.WindowCenter
        width = ds.WindowWidth
        # Handle multi-valued tags
        if isinstance(center, pydicom.multival.MultiValue):
            center = center[0]
            width = width[0]
        low = center - width/2
        high = center + width/2
        arr = np.clip(arr, low, high)
    arr = (arr - arr.min()) / (arr.max() - arr.min())  # normalize 0–1
    arr = (arr * 255).astype(np.uint8)                  # scale to 0–255
    return arr


In [9]:
from pathlib import Path
from PIL import Image
import os

IMAGE_EXTENSIONS = {'.dcm'}

def read_images(root_dir):
    root = Path(root_dir)
    for path in root.rglob('*'):
        if path.suffix.lower() in IMAGE_EXTENSIONS:
            img = read_dicom(path)
            yield path, img

# Usage
# parent_dir="ReadableVersion"
for path, img in read_images('Yarışma 2.aşama veri seti kümesi'):
    current_dir = os.path.join("ReadableVersion",str(path)[:str(path).rfind("\\")])
    if(os.path.exists(current_dir) == False):
        os.makedirs(current_dir, exist_ok=True)
        print(f"Directory created: {current_dir}")
    img = Image.fromarray(img)
    img.save(os.path.join("ReadableVersion",str(path)[:-4] + ".png"))
    print(f"{path}: size={img.size}")

Yarışma 2.aşama veri seti kümesi\Vaka_300571\CT\Seri2\50338780.1.103.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\CT\Seri2\50338780.1.84.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri2\50009482.1.45.dcm: size=(256, 256)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri2\50009482.1.65.dcm: size=(256, 256)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri4\50009482.3.49.dcm: size=(256, 256)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri4\50009482.3.69.dcm: size=(256, 256)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri5\50009482.4.13.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri5\50009482.4.9.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300601\CT\Seri2\50035136.1.110.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300601\CT\Seri2\50035136.1.86.dcm: size=(512, 512)
Yarışma 2.aşama veri seti kümesi\Vaka_300601\MR\Seri1\50427468.0.16.dcm: size=(256, 256)
Yarışma 2.aşama veri

In [12]:
data

[{'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.31.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.32.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.0.33.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.5.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.6.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.1.7.dcm',
  'Modality': 'MR',
  'LessionTypeName': 'HiperakutAkut'},
 {'PatientId': 'Vaka_302861',
  'StudyId': '50479238',
  'ImageId': '50479238.3.5.dcm',
  'Modality': 'MR',
  'Less

In [ ]:
IMAGE_EXTENSIONS_V = {'.png'}

def read_paths(root_dir):
    root = Path(root_dir)
    for path in root.rglob('*'):
        if path.suffix.lower() in IMAGE_EXTENSIONS_V:
            yield path

# names=[]
for item in data:
    if(os.path.exists(os.path.join("ReadableVersion/Yarışma 2.aşama veri seti kümesi",item["PatientId"])) == False):
        print(f"Image not found: {item['ImageId']}")
        continue
    img_path = os.path.join("ReadableVersion/Yarışma 2.aşama veri seti kümesi",item["PatientId"])
    for path in read_paths(img_path):
        # print(f"Checking {path} for {item['ImageId']}")
        if(path.name == item["ImageId"].replace(".dcm", ".png")):
            lession_type = item['LessionTypeName']
            os.rename(path, os.path.join(path.parent, path.name.replace(".png", f"_{lession_type}.png")))
            print(f"Renaming {path.name.replace('.png', f'_{lession_type}.png')}")
    

Renaming 50479238.0.31_HiperakutAkut.png
Renaming 50479238.0.32_HiperakutAkut.png
Renaming 50479238.0.33_HiperakutAkut.png
Renaming 50479238.1.5_HiperakutAkut.png
Renaming 50479238.1.6_HiperakutAkut.png
Renaming 50479238.1.7_HiperakutAkut.png
Renaming 50479238.3.5_HiperakutAkut.png
Renaming 50479238.3.6_HiperakutAkut.png
Renaming 50479238.3.7_HiperakutAkut.png
Renaming 50021760.1.32_NormalKronik.png
Renaming 50021760.1.39_NormalKronik.png
Renaming 50021760.1.42_NormalKronik.png
Renaming 50021760.2.8_NormalKronik.png
Renaming 50021760.2.15_NormalKronik.png
Renaming 50021760.2.18_NormalKronik.png
Renaming 50412696.1.7_NormalKronik.png
Renaming 50412696.1.12_NormalKronik.png
Renaming 50412696.1.14_NormalKronik.png
Renaming 50412696.1.29_NormalKronik.png
Renaming 50412696.1.34_NormalKronik.png
Renaming 50412696.1.36_NormalKronik.png
Renaming 50412696.2.7_NormalKronik.png
Renaming 50412696.2.12_NormalKronik.png
Renaming 50412696.2.14_NormalKronik.png
Renaming 50215336.1.9_HiperakutAkut.png


In [40]:

i=0
for item in data:
    if(item["LessionTypeName"] != "NormalKronik"):
        i+=1

i

2572

In [37]:
# os.path.exists("ReadableVersion\\Yarışma 2.aşama veri seti kümesi\\Vaka_302861\\MR\\Seri1\\")

In [ ]:

#makes sure there are no multiple labeled images
# for name in names:
#     code= name.split("_")[0]
#     length = len(code)
#     for name_ in names:
#         if(name_[:length]==code and len(name_.split("_")[0])==length and  name_ != name):
#             print(f"{name} and {name_}")

In [36]:
# names